# To Begin With...

In this first notebook, our goal was to discover the data. Moreover we did some processing for the problem.

In [1]:
%%configure
{"conf": {
    "spark.app.name": "group100_final"
}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9130,application_1589299642358_3694,pyspark,idle,Link,Link,
9171,application_1589299642358_3738,pyspark,idle,Link,Link,
9188,application_1589299642358_3755,pyspark,idle,Link,Link,
9192,application_1589299642358_3759,pyspark,idle,Link,Link,
9204,application_1589299642358_3771,pyspark,idle,Link,Link,
9205,application_1589299642358_3772,pyspark,idle,Link,Link,
9206,application_1589299642358_3773,pyspark,idle,Link,Link,
9216,application_1589299642358_3783,pyspark,idle,Link,Link,
9217,application_1589299642358_3784,pyspark,idle,Link,Link,
9225,application_1589299642358_3792,pyspark,idle,Link,Link,


### Start Spark

In [7]:
# Initialization

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
username = 'mjouve'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
from pyspark.sql.functions import udf
import geopy.distance

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Read the [SBB actual data](https://opentransportdata.swiss/en/dataset/istdaten) in ORC format

In [10]:
sbb = spark.read.orc('/data/sbb/orc/istdaten')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
sbb.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- betriebstag: string (nullable = true)
 |-- fahrt_bezeichner: string (nullable = true)
 |-- betreiber_id: string (nullable = true)
 |-- betreiber_abk: string (nullable = true)
 |-- betreiber_name: string (nullable = true)
 |-- produkt_id: string (nullable = true)
 |-- linien_id: string (nullable = true)
 |-- linien_text: string (nullable = true)
 |-- umlauf_id: string (nullable = true)
 |-- verkehrsmittel_text: string (nullable = true)
 |-- zusatzfahrt_tf: string (nullable = true)
 |-- faellt_aus_tf: string (nullable = true)
 |-- bpuic: string (nullable = true)
 |-- haltestellen_name: string (nullable = true)
 |-- ankunftszeit: string (nullable = true)
 |-- an_prognose: string (nullable = true)
 |-- an_prognose_status: string (nullable = true)
 |-- abfahrtszeit: string (nullable = true)
 |-- ab_prognose: string (nullable = true)
 |-- ab_prognose_status: string (nullable = true)
 |-- durchfahrt_tf: string (nullable = true)

In [12]:
sbb.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------------+------------+-------------+--------------+----------+---------+-----------+---------+-------------------+--------------+-------------+-------+-----------------+------------+-----------+------------------+----------------+-----------+------------------+-------------+
|betriebstag|   fahrt_bezeichner|betreiber_id|betreiber_abk|betreiber_name|produkt_id|linien_id|linien_text|umlauf_id|verkehrsmittel_text|zusatzfahrt_tf|faellt_aus_tf|  bpuic|haltestellen_name|ankunftszeit|an_prognose|an_prognose_status|    abfahrtszeit|ab_prognose|ab_prognose_status|durchfahrt_tf|
+-----------+-------------------+------------+-------------+--------------+----------+---------+-----------+---------+-------------------+--------------+-------------+-------+-----------------+------------+-----------+------------------+----------------+-----------+------------------+-------------+
| 03.09.2018|80:06____:17010:000|   80:06____|           DB|   DB Regio AG|       Zug|    17010|    

### Read the station list data [BFKOORD_GEO](https://opentransportdata.swiss/en/cookbook/hafas-rohdaten-format-hrdf/#Abgrenzung)

In [13]:
metadata = spark.read.csv('/data/sbb/stations/bfkoordgeo.csv', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
metadata.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------+---------+------+---------+
|StationID|Longitude| Latitude|Height|   Remark|
+---------+---------+---------+------+---------+
|  0000002|26.074412|44.446770|     0|Bucuresti|
|  0000003| 1.811446|50.901549|     0|   Calais|
+---------+---------+---------+------+---------+
only showing top 2 rows

# Stops data

In [16]:
stops = spark.read.orc('/data/sbb/timetables/orc/stops/')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
stops.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------+----------------+----------------+-------------+--------------+
|stop_id|   stop_name|        stop_lat|        stop_lon|location_type|parent_station|
+-------+------------+----------------+----------------+-------------+--------------+
|1322000|    Altoggio|46.1672513851495|  8.345807131427|             |              |
|1322001|Antronapiana| 46.060121674738|8.11361957990831|             |              |
+-------+------------+----------------+----------------+-------------+--------------+
only showing top 2 rows

# Stop times data

In [18]:
stop_times = spark.read.csv('/data/sbb/timetables/csv/stop_times/2019/05/14/stop_times.txt', sep=',', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
stop_times.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+-----------+-------------+-----------+-------------+
|             trip_id|arrival_time|departure_time|    stop_id|stop_sequence|pickup_type|drop_off_type|
+--------------------+------------+--------------+-----------+-------------+-----------+-------------+
|1.TA.1-1-B-j19-1.1.R|    04:20:00|      04:20:00|8500010:0:3|            1|          0|            0|
|1.TA.1-1-B-j19-1.1.R|    04:24:00|      04:24:00|8500020:0:3|            2|          0|            0|
+--------------------+------------+--------------+-----------+-------------+-----------+-------------+
only showing top 2 rows

# Trips data

In [20]:
trips = spark.read.orc('/data/sbb/timetables/orc/trips')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
trips.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+--------------------+------------------+---------------+------------+
|   route_id|service_id|             trip_id|     trip_headsign|trip_short_name|direction_id|
+-----------+----------+--------------------+------------------+---------------+------------+
|1-1-C-j19-1|  TA+b0001|5.TA.1-1-C-j19-1.3.R|Zofingen, Altachen|            108|           1|
|1-1-C-j19-1|  TA+b0001|7.TA.1-1-C-j19-1.3.R|Zofingen, Altachen|            112|           1|
+-----------+----------+--------------------+------------------+---------------+------------+
only showing top 2 rows

# Calendar data

In [23]:
calendar = spark.read.orc('/data/sbb/timetables/orc/calendar')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
calendar.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------+---------+--------+------+--------+------+
|service_id|monday|tuesday|wednesday|thursday|friday|saturday|sunday|
+----------+------+-------+---------+--------+------+--------+------+
|  TA+b0nx9|  true|   true|     true|    true|  true|   false| false|
|  TA+b03bf|  true|   true|     true|    true|  true|   false| false|
+----------+------+-------+---------+--------+------+--------+------+
only showing top 2 rows

# Routes data

In [26]:
routes = spark.read.orc('/data/sbb/timetables/orc/routes')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
routes.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+----------------+---------------+----------+----------+
|   route_id|agency_id|route_short_name|route_long_name|route_desc|route_type|
+-----------+---------+----------------+---------------+----------+----------+
|11-40-j19-1|      801|             040|               |       Bus|       700|
|11-61-j19-1|     7031|             061|               |       Bus|       700|
+-----------+---------+----------------+---------------+----------+----------+
only showing top 2 rows

# 1. Zurichs stops - Jump to last line to load it directly

_The first step to process the data is to keep only the stops that are at most 15km from the Zürich HB station._

_Let's first obtain the longitude and latitude of the `Zürich HB` station._

In [28]:
metadata.filter(metadata.Remark == 'Zürich HB').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------+---------+------+---------+
|StationID|Longitude| Latitude|Height|   Remark|
+---------+---------+---------+------+---------+
|  8503000| 8.540192|47.378177|   408|Zürich HB|
+---------+---------+---------+------+---------+

In [29]:
import geopy.distance

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
from pyspark.sql.functions import udf
@udf("float")
def distance_from_zurich_HB_udf(lat, lon):
    """
    Given the latitude and the longitude of a point, compute its distance in km from the Zürich HB station
    """
    
    zurich_HB_point = (47.37817, 8.540192)
    
    return geopy.distance.vincenty(zurich_HB_point, (lat, lon)).km

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
zurich_stops = stops.filter(distance_from_zurich_HB_udf(stops.stop_lat, stops.stop_lon) <= 15)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
zurich_stops.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+-------------+--------------+
|stop_id|           stop_name|        stop_lat|        stop_lon|location_type|parent_station|
+-------+--------------------+----------------+----------------+-------------+--------------+
|8500926|Oetwil a.d.L., Sc...|47.4236270123012| 8.4031825286317|             |              |
|8502186|Dietikon Stoffelbach|47.3934058321612|8.39894248049007|             |      8502186P|
+-------+--------------------+----------------+----------------+-------------+--------------+
only showing top 2 rows

_After careful examination of the dataframe, we realized that some stops have multiple entries. The only way to keep only one entry per stop is to keep only one entry per name. Regarding the small differences in latitude and longitute between two entries having the same name, we consider it as not relevant_

In [33]:
zurich_stops = zurich_stops.dropDuplicates(['stop_name'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

_Finally the last observation that can be made is that some `stop_id` have some additional information if for exemple there multiple stops in the same station written with `:X:Y`. The main entry for a station also usually contains the character `P` but it so not always the case. Now that we have one entry per name, we can discard those extra characters._

_We will also remove the `location_type` and the `parent_station` columns._

In [34]:
@udf
def remove_useless_stop_data(stop_id):
    if ':' in stop_id:
        stop_id = stop_id.split(':')[0]
    if 'P' in stop_id:
        stop_id = stop_id.replace('P', '')
        
    return stop_id

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
zurich_stops = zurich_stops.select(remove_useless_stop_data(zurich_stops.stop_id).alias('stop_id'),
                                   zurich_stops.stop_name,
                                   zurich_stops.stop_lat,
                                   zurich_stops.stop_lon)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
zurich_stops.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+
|stop_id|           stop_name|        stop_lat|        stop_lon|
+-------+--------------------+----------------+----------------+
|8587973|Erlenbach ZH, Im ...| 47.305781467323|8.59663472509377|
|8576152|  Kloten, Kirchgasse|47.4529136309309| 8.5843457720053|
+-------+--------------------+----------------+----------------+
only showing top 2 rows

_We will save the above dataframe._

In [37]:
# already saved
# zurich_stops.write.format("orc").mode("overwrite").save("/user/{}/zurich_stops.orc".format(username))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
zurich_stops = spark.read.orc("/user/{}/zurich_stops.orc".format(username)).cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
zurich_stops.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+
|stop_id|           stop_name|        stop_lat|        stop_lon|
+-------+--------------------+----------------+----------------+
|8590480|Adliswil, Riferts...| 47.310118327592|8.53154279959741|
|8576271|Dällikon, Seitena...|47.4387766700117|8.44470266106948|
+-------+--------------------+----------------+----------------+
only showing top 2 rows

# 2. Calendar - Service available at least from monday to friday

_We will keep the services that are available at least from monday to friday._

In [40]:
calendar_filtered = calendar.filter(calendar.monday &
                                    calendar.tuesday &
                                    calendar.wednesday &
                                    calendar.thursday &
                                    calendar.friday).cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
calendar_filtered.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------+---------+--------+------+--------+------+
|service_id|monday|tuesday|wednesday|thursday|friday|saturday|sunday|
+----------+------+-------+---------+--------+------+--------+------+
|  TA+b0nx9|  true|   true|     true|    true|  true|   false| false|
|  TA+b03bf|  true|   true|     true|    true|  true|   false| false|
+----------+------+-------+---------+--------+------+--------+------+
only showing top 2 rows

# 3. Obtain stop_times for Zürich with available services - Jump to last line to load it directly

_First, as the `stop_times` dataframes contains a `stop_id` field, we must remove the useless information as we did for the `stops` dataframe in order to properly join them. We also remove the `pickup_type` and `drop_off_type` as they are note usefull for now._

In [42]:
# This one is more suited than the one used for zurich stops and uses less ressources
@udf
def remove_useless_stop_times_data(stop_id):
    if ':' in stop_id:
        return stop_id.split(':')[0]
    return stop_id

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

_We will now get the `trip_id` that are available at least from monday to friday._

In [43]:
services_available = calendar_filtered.select('service_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
trips_available = services_available.join(trips, on='service_id').select('trip_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
zurich_stop_ids = zurich_stops.select('stop_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

_Finally we keep only the trips available and the one from Zürich._

In [46]:
stop_times = stop_times.select(stop_times.trip_id,
                               stop_times.arrival_time,
                               stop_times.departure_time,
                               remove_useless_stop_times_data(stop_times.stop_id).alias('stop_id'),
                               stop_times.stop_sequence).cache()

stop_times_zurich = zurich_stop_ids.join(stop_times, on='stop_id')
stop_times_filtered = trips_available.join(stop_times_zurich, on='trip_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
stop_times_filtered.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------+------------+--------------+-------------+
|             trip_id|stop_id|arrival_time|departure_time|stop_sequence|
+--------------------+-------+------------+--------------+-------------+
|1005.TA.26-131-j1...|8503855|    16:14:00|      16:14:00|            1|
|1005.TA.26-131-j1...|8589111|    16:15:00|      16:15:00|            2|
+--------------------+-------+------------+--------------+-------------+
only showing top 2 rows

_We will save the above dataframe._

In [49]:
# already saved
# stop_times_filtered.write.format("orc").mode("overwrite").save("/user/{}/stop_times_filtered.orc".format(username))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
stop_times_filtered = spark.read.orc("/user/{}/stop_times_filtered.orc".format(username))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
stop_times_filtered.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------+------------+--------------+-------------+
|             trip_id|stop_id|arrival_time|departure_time|stop_sequence|
+--------------------+-------+------------+--------------+-------------+
|1.TA.26-10-B-j19-...|8503051|    05:09:00|      05:09:00|            1|
|1.TA.26-10-B-j19-...|8503052|    05:11:00|      05:11:00|            2|
+--------------------+-------+------------+--------------+-------------+
only showing top 2 rows

# 4 Compute reachable stations pairs - Jump at the end to directly load it

_Let's obtain the stops that are reachable from one other by walking (<500m flying distance)._

In [52]:
zurich_stops.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+
|stop_id|           stop_name|        stop_lat|        stop_lon|
+-------+--------------------+----------------+----------------+
|8590480|Adliswil, Riferts...| 47.310118327592|8.53154279959741|
|8576271|Dällikon, Seitena...|47.4387766700117|8.44470266106948|
+-------+--------------------+----------------+----------------+
only showing top 2 rows

In [53]:
zurich_stops_1 = zurich_stops.select(zurich_stops.stop_id.alias('id_1'),
                      zurich_stops.stop_name.alias('name_1'),
                      zurich_stops.stop_lat.alias('lat_1'),
                      zurich_stops.stop_lon.alias('lon_1'))

zurich_stops_2 = zurich_stops.select(zurich_stops.stop_id.alias('id_2'),
                      zurich_stops.stop_name.alias('name_2'),
                      zurich_stops.stop_lat.alias('lat_2'),
                      zurich_stops.stop_lon.alias('lon_2'))

zurich_stops_pair = zurich_stops_1.crossJoin(zurich_stops_2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
zurich_stops_pair = zurich_stops_pair.filter(zurich_stops_pair.id_1 != zurich_stops_pair.id_2).distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
@udf
def distance(lat_1, lon_1, lat_2, lon_2):
    return geopy.distance.vincenty((lat_1, lon_1), (lat_2, lon_2)).m

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
zurich_stops_pair = zurich_stops_pair.withColumn('distance', distance(zurich_stops_pair.lat_1, 
                                                                      zurich_stops_pair.lon_1, 
                                                                      zurich_stops_pair.lat_2, 
                                                                      zurich_stops_pair.lon_2))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
zurich_reachable_pair = zurich_stops_pair.filter(zurich_stops_pair.distance <= 500).select(zurich_stops_pair.id_1,
                                                                                  zurich_stops_pair.name_1,
                                                                                  zurich_stops_pair.id_2,
                                                                                  zurich_stops_pair.name_2,
                                                                                  zurich_stops_pair.distance)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

_We can save the `zurich_reachable_pair` as well as the `calendar_filtered` dataframes._

In [58]:
# already saved
# zurich_reachable_pair.write.format("orc").mode("overwrite").save("/user/{}/zurich_reachable_pair.orc".format(username))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
zurich_reachable_pair = spark.read.orc("/user/{}/zurich_reachable_pair.orc".format(username))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
zurich_reachable_pair.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+-------+--------------------+------------------+
|   id_1|              name_1|   id_2|              name_2|          distance|
+-------+--------------------+-------+--------------------+------------------+
|8503372|Effretikon, Watts...|8575921|Effretikon, Linde...|422.70473682950103|
|8591406|Zürich, Universit...|8591276|   Zürich, Milchbuck|264.38640306079475|
|8591329|Zürich, Saalsport...|8591245|     Zürich, Laubegg|268.45762594909314|
|8590814|Thalwil, Aubrigst...|8590825|Thalwil, Rainstrasse| 142.4267701370411|
|8591045|Zürich, Appenzell...|8591121|Zürich, Eschergutweg|  349.486525106936|
+-------+--------------------+-------+--------------------+------------------+
only showing top 5 rows

# Next step: 

_Now that we discovered the data and made a first processing, the goal was to implement a first static journey planning algorithm, that does not take into consideration the probability._

_The different steps to build our route planning algorithm can be observed in the `route_planning.ipynb` notebook._